In [1]:
from dotenv import load_dotenv

load_dotenv()

from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_text_splitters.base import Language
from langchain_community.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.chat_models import ChatOpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import (
    HumanMessage,AIMessage
)

import gradio as gr
import json
import openai

c:\Users\oksk1\venv\lc_rag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
llm = ChatOpenAI(temperature=0, model="gpt-4")

# llm
# llm=ChatOpenAI(
#     streaming=True,
#     callbacks=[StreamingStdOutCallbackHandler()],
#     model="gpt-3.5-turbo-0125",
#     temperature=0,
# )

c:\Users\oksk1\venv\lc_rag\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [3]:
# 해당 부분은 이미 임베딩된 데이터를 로드하는 부분이다.

vectorstore = Chroma(persist_directory='../data/chroma_db', embedding_function=OpenAIEmbeddings())

c:\Users\oksk1\venv\lc_rag\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [4]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 1})
#prompt = hub.pull("rlm/rag-prompt")
prompt = hub.pull("hwchase17/openai-tools-agent")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [5]:
qa_interface = RetrievalQA.from_chain_type(llm=llm,
                                           chain_type="stuff",
                                           retriever=retriever,
                                           chain_type_kwargs={
                                            "verbose": True,
                                           },
                                           return_source_documents=True)

In [8]:
with gr.Blocks() as rag_tester:
    gr.HTML("법률 사례기반 챗봇")

    with gr.Row():
        with gr.Column(scale=1):
            chatbot = gr.Chatbot()
            msg = gr.Textbox(placeholder="법률 관련 질문을 입력하세요.")
        with gr.Column(scale=1):
            gr.HTML("검색 문서")
            source = gr.Textbox()

    def userHandler(user_message, history):
        #print("@@ history:", history)
        return "", history + [[user_message, None]]

    def botHandler(history):
        qa_result = qa_interface(history[-1][0])

        history[-1][1] = qa_result["result"]
        source_text = ""
        for ix, doc in enumerate(qa_result['source_documents']):
            source_text += f"## 검색문서 {ix+1}\n```\n{doc.page_content}\n```\n\n"

        # for i in range(len(history)):
        #     print(f"### {i}번째\nQuestion: {history[i][0]}\nAnswer: {history[i][1]}")
        print("## history:", history)

        return history, source_text

    msg.submit(userHandler, [msg, chatbot], [msg, chatbot], queue=False).then(
        botHandler, chatbot, [chatbot, source]
    )

rag_tester.launch(debug=True, share=True)

@@ msg: <gradio.components.textbox.Textbox object at 0x000001BB7A67FB10>
Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
"안녕하세요 저는 휴대폰 매장 대표 입니다, 직원이 돈이 없엇어 퀵비를 안줫다구 합ㅈ니다. 제가 그 거땜에 퀵기사님 상대로 저 를 고소를 햇다구합니다.저는 드릴러고 햇는데 자꾸 안받는다구 햇어고 그래서 매장에서 도 매번 오서는데 제가 드릴러고 햇어는데 자꾸 안받는다 그래서 매장 근처 돌면서 동영상 도찍으면서 돈을자체 안받는다구 하섯고 그래서 저 상대러 고소 햇다구 합니다, 그래서 오늘 경찰 출석인데 제가 늦쳣습니다 낼 경찰 출석인데 변호사님들이랑 같이 동해 하고 싶은데 비용이 문제라서 ㅠㅠㅠ 그래서 이렇게 햇습니다 낼 변호사님이랑 동행 할수잇으가요",,불구속 수사중인 경우 국선변호사 제도가 없으며 변호사 선임을 원하면 개인적으로 선임하여야 합니다.
Human: 안녕하세요

> Finished chain.

> Finished chain.
### 0번째
Question: 안녕하세요
Answer: 안녕하세요, 어떻게 도와드릴까요?


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know

In [6]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor
from langchain.prompts import MessagesPlaceholder

In [19]:
# llm
llm=ChatOpenAI(
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model="gpt-3.5-turbo-0125",
    temperature=0
)

# prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            당신은 법률기반 질의응답에 기반해 사용자의 질문에 답변하는 챗봇입니다.
            반드시 전달받은 context에 기반해 답변해야 하며,
            전달받은 context로 답변할 수 없을 시 답변을 절대 하면 안 됩니다.
            Lets' think step by step.
            """
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad") # 메세지 리스트를 전달하는 공간
    ]
)

retriever_tool = create_retriever_tool(
    retriever,
    "law_qa_search",
    "법률에 대한 질문이라면 반드시 이 tool을 사용하세요.",
)


# search
search = TavilySearchResults(max_results=1)

# tools
# tools = [retriever_tool]
tools = [retriever_tool, search]


# # 언어체인
# combine_docs_chain = create_stuff_documents_chain(llm, prompt)
# # 검색체인
# retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

result1 = agent_executor.invoke({"input": "근로기준법상 근로자에는 정규직만 들어가는 건가요?"})
result2 = agent_executor.invoke({"input": "연예인 뉴진스에 대해 설명해주세요."})



> Entering new AgentExecutor chain...

Invoking: `law_qa_search` with `{'query': '근로기준법 근로자 정의'}`


근무하는 한국인 노동자에게는 「근로기준법」및 「산업재해보상보험법」이 적용됩니다(산업재해보상보험법 제122조).다만, 상시 5인 이상의 근로자를 사용하는 사업 또는 사업장이라도 다음의 경우에는 「근로기준법」이 적용되지 않습니다.첫째, 동거의 친족만을 사용하는 사업은 종속적 근로관계를 인정할 수 없으니 친족상호간에 누가 사용자이고 노동자인지를 형식적으로 구별하기 곤란하기 때문에 법적용에서 제외하고 있습니다. 그러나 「민법」제777조에 의한 법률상의 친족범위(8촌 이내의 혈족, 4촌 이내의 인척, 배우자)밖에 있는 친족을 근로자로 사용하는 경우에는 「근로기준법」이 적용됩니다.둘째, 가사사용인은 제외됩니다. 가사사용인이란 일반가정의 가정부, 파출부 등 가사에 종사하는 자를 말합니다. 가사사용인과의 관계는 주로 개인의 사생활과 관련되어 있고 근무시간이나 임금규제를 통하여 국가의 감독행정이 미치기 어렵기 때문입니다.셋째, 특별법에 의한 적용의 예외로서 선원의 경우는 해상노동의 특이성을 고려하여 선원법이 적용되며, 국가공무원 및 지방공무원은 특별규정(예：공무원연금법, 공무원보수규정 등)이 있는 경우에는 「근로기준법」의 적용이 제한되며, 사립학교교직원은 「사립학교법」에 특별히 「근로기준법」의 적용을 배제한다는 규정이 없는 한 「근로기준법」이 적용됩니다.또한 판례는 “근로기준법 제10조(현행 근로기준법 제11조) 본문에서 ‘상시 5인 이상의 근로자를 사용하는 사업 또는 사업장’이라고 함은 근로자 수가 때때로 5인 미만이 되는 경우가 있어도 상태적으로 5인 이상이 되는 경우에는 이에 해당하고, 나아가 당해 사업장에 계속 근무하는 근로자뿐 아니라 그때 그때의 필요에 의하여 사용하는 일용근로자도 포함하는 것이다.”라고 하였습니다(대법원 1997. 11. 28. 선고 97다28971 판결, 2000. 3